# Chapter 4 用張量表示現實中的資料

## Session 4.1 圖片資料

## Session 4.2 3D立體圖片資料

In [4]:
import imageio
import torch
dir_path = "../F1388_Code/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr =imageio.volread(dir_path, 'DICOM')
vol_arr.shape

#(99, 512, 512)通道軸被省略掉了 
#99 張數
#512 512 為尺寸

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [6]:
# Pytorch要求個軸排列順序 N x C x H x W
vol = torch.from_numpy(vol_arr).float() #先把資料轉成浮點數張量
vol = torch.unsqueeze(vol, 1 ) #在第2軸位置增加插入一個通道軸，dim=1
vol = torch.unsqueeze(vol, 2 ) #在第3軸位置增加插入一個深度軸，dim=1
vol.shape

torch.Size([99, 1, 1, 512, 512])